In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Scraping web data to extract Calgary city data

In [81]:
#Scraping web data
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'
html_data=requests.get(url).text

In [83]:
soup = BeautifulSoup(html_data, "html5lib")

In [84]:
print(soup.title.text)

List of postal codes of Canada: T - Wikipedia


In [85]:
for link in soup.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

Inner Text: 
Title: None
href: None
Inner Text: Jump to navigation
Title: None
href: #mw-head
Inner Text: Jump to search
Title: None
href: #searchInput
Inner Text: postal codes in Canada
Title: Postal codes in Canada
href: /wiki/Postal_codes_in_Canada
Inner Text: Alberta
Title: Alberta
href: /wiki/Alberta
Inner Text: Canada Post
Title: Canada Post
href: /wiki/Canada_Post
Inner Text: [1]
Title: None
href: #cite_note-1
Inner Text: applications
Title: Mobile application software
href: /wiki/Mobile_application_software
Inner Text: smartphones
Title: Smartphones
href: /wiki/Smartphones
Inner Text: iPhone
Title: IPhone
href: /wiki/IPhone
Inner Text: BlackBerry
Title: BlackBerry
href: /wiki/BlackBerry
Inner Text: [2]
Title: None
href: #cite_note-2
Inner Text: CD-ROMs
Title: CD-ROM
href: /wiki/CD-ROM
Inner Text: Alberta
Title: Alberta
href: /wiki/Alberta
Inner Text: FSAs
Title: Canadian postal code
href: /wiki/Canadian_postal_code#Forward_sortation_areas
Inner Text: edit
Title: Edit section: A

href: /wiki/Stony_Plain,_Alberta
Inner Text: Alberta
Title: Alberta
href: /wiki/Alberta
Inner Text: FSAs
Title: Canadian postal code
href: /wiki/Canadian_postal_code#Forward_sortation_areas
Inner Text: edit
Title: Edit section: Alberta - 157 FSAs
href: /w/index.php?title=List_of_postal_codes_of_Canada:_T&action=edit&section=3
Inner Text: edit
Title: Edit section: Rural
href: /w/index.php?title=List_of_postal_codes_of_Canada:_T&action=edit&section=4
Inner Text: Abee
Title: Abee, Alberta
href: /wiki/Abee,_Alberta
Inner Text: Ardmore
Title: Ardmore, Alberta
href: /wiki/Ardmore,_Alberta
Inner Text: Ashmont
Title: Ashmont, Alberta
href: /wiki/Ashmont,_Alberta
Inner Text: Atmore
Title: Atmore, Alberta
href: /wiki/Atmore,_Alberta
Inner Text: Bellis
Title: Bellis, Alberta
href: /wiki/Bellis,_Alberta
Inner Text: Bon Accord
Title: Bon Accord, Alberta
href: /wiki/Bon_Accord,_Alberta
Inner Text: Boyle
Title: Boyle, Alberta
href: /wiki/Boyle,_Alberta
Inner Text: Boyne Lake
Title: Boyne Lake, Albert

In [86]:
calgary_tables = soup.find_all('table')
for index, table in enumerate(calgary_tables):
    if ("Postal Code" in str(table)):
        table_index = index
print(table_index)

1


In [87]:
calgary_env = pd.DataFrame(columns=["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"])
for row in calgary_tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Postal_Code = col[0].text.replace('\n', ' ').strip()
        Borough = col[1].text.replace('\n', ' ').strip()
        Neighborhood = col[2].text.replace('\n', ' ').strip()
        Latitude = col[3].text.replace('\n', ' ').strip()
        Longitude = col[4].text.replace('\n', ' ').strip()
        
        calgary_env = calgary_env.append({"Postal Code":Postal_Code, "Borough":Borough, "Neighborhood":Neighborhood, "Latitude":Latitude, "Longitude":Longitude}, ignore_index = True)

calgary_env

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
5,T6A,Edmonton,North Capilano,53.5483,-113.408
6,T7A,Drayton Valley,Not assigned,53.2165,-114.9893
7,T8A,Sherwood Park,West Sherwood Park,53.519,-113.3216
8,T9A,Wetaskiwin,Not assigned,52.9741,-113.3646
9,T1B,Medicine Hat,South Medicine Hat,50.0172,-110.651


In [88]:
#shape of dataset
calgary_env.shape

(180, 5)

### Define Foursquare Credentials and Version

In [89]:
CLIENT_ID = 'UTKPC0DTHOBPJHPIQJLWAZFGQYJ1HGDYKGJLLGRE5UG3BXEO' # your Foursquare ID
CLIENT_SECRET = '433UDLMCRC2HUFN5LWWYUL5QPHADPCRNIN25MVHYVF0NYHUB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UTKPC0DTHOBPJHPIQJLWAZFGQYJ1HGDYKGJLLGRE5UG3BXEO
CLIENT_SECRET:433UDLMCRC2HUFN5LWWYUL5QPHADPCRNIN25MVHYVF0NYHUB


In [31]:
from geopy.geocoders import Nominatim

### Extracting Boroughs that contain "Calgary"

In [183]:
calgary_data = calgary_env[calgary_env['Borough'].str.contains("Calgary")].reset_index(drop=True)
calgary_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
1,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
2,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786
3,T3B,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",51.0809,-114.1616
4,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001
5,T3C,Calgary,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",51.0388,-114.098
6,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.0632,-114.0614
7,T3E,Calgary,"Lakeview, Glendale, Killarney, Glamorgan",51.0227,-114.1342
8,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria...",51.0415,-114.0599
9,T3G,Calgary,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",51.1147,-114.1796


In [41]:
#Removing row 27 with "Not Assigned" Data

In [184]:
calgary_data = calgary_data.drop([27], axis=0)
calgary_data.tail(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
24,T2R,Calgary,"Connaught, West Victoria Park",51.0426,-114.0791
25,T3R,Calgary,Northwest Calgary,51.1497,-114.2695
26,T2S,Calgary,"Elbow Park, Britannia, Parkhill, Mission",51.0171,-114.0812
28,T2T,Calgary,South Calgary (Altadore / Bankview / Richmond),51.0316,-114.0994
29,T2V,Calgary,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",50.9909,-114.074
30,T2W,Calgary,"Braeside, Cedarbrae, Woodbine",50.9604,-114.1001
31,T2X,Calgary,"Midnapore, Sundance",50.9204,-114.0674
32,T1Y,Calgary,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015
33,T2Y,Calgary,"Millrise, Somerset, Bridlewood, Evergreen",50.9093,-114.0721
34,T2Z,Calgary,"Douglas Glen, McKenzie Lake, Copperfield, East...",50.9023,-113.9873


In [40]:
calgary_data.reset_index(drop=True)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
1,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
2,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786
3,T3B,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",51.0809,-114.1616
4,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001
5,T3C,Calgary,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",51.0388,-114.098
6,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.0632,-114.0614
7,T3E,Calgary,"Lakeview, Glendale, Killarney, Glamorgan",51.0227,-114.1342
8,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria...",51.0415,-114.0599
9,T3G,Calgary,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",51.1147,-114.1796


In [92]:
address = 'Calgary'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary are 51.0534234, -114.0625892.


In [93]:
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(calgary_data['Latitude'], calgary_data['Longitude'], calgary_data['Borough'], calgary_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

### Exploring Neighborhoods in Calgary

In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [95]:
#Creating Calgary venues
calgary_venues = getNearbyVenues(names=calgary_data['Neighborhood'],
                                   latitudes=calgary_data['Latitude'],
                                   longitudes=calgary_data['Longitude']
                                  )

Penbrooke Meadows, Marlborough
Dalhousie, Edgemont, Hamptons, Hidden Valley
Forest Lawn, Dover, Erin Woods
Montgomery, Bowness, Silver Springs, Greenwood
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Bridgeland, Greenview, Zoo, YYC
Lakeview, Glendale, Killarney, Glamorgan
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge
Highfield, Burns Industrial
Discovery Ridge, Signal Hill, West Springs, Christie Estates, Patterson, Cougar Ridge
Queensland, Lake Bonavista, Willow Park, Acadia
Martindale, Taradale, Falconridge, Saddle Ridge
Thorncliffe, Tuxedo Park
Sandstone, MacEwan Glen, Beddington, Harvest Hills, Coventry Hills, Panorama Hills
Brentwood, Collingwood, Nose Hill
Tuscany, Scenic Acres
Mount Pleasant, Capitol Hill, Banff Trail
Cranston, Auburn Bay, Mahogany
Kensington, Westmont, Parkdale, University
Northeast Calgary
City Centre, Calgary To

In [96]:
#Size of venues dataframe
print(calgary_venues.shape)
calgary_venues.head()

(332, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Penbrooke Meadows, Marlborough",51.049680,-113.964320,Bearcat General Contracting,51.047779,-113.968599,Construction & Landscaping
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Petro-Canada,51.128068,-114.138057,Gas Station
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Edgemont City,51.126473,-114.138997,Asian Restaurant
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Friends Cappuccino Bar & Bake Shop,51.126370,-114.138676,Café
4,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Mac's,51.128309,-114.137902,Convenience Store


In [97]:
#checking how many venues were returned for each neighborhood using group_by
calgary_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Braeside, Cedarbrae, Woodbine",9,9,9,9,9,9
"Brentwood, Collingwood, Nose Hill",1,1,1,1,1,1
"Bridgeland, Greenview, Zoo, YYC",23,23,23,23,23,23
"City Centre, Calgary Tower",27,27,27,27,27,27
"Connaught, West Victoria Park",40,40,40,40,40,40
"Cranston, Auburn Bay, Mahogany",3,3,3,3,3,3
"Dalhousie, Edgemont, Hamptons, Hidden Valley",4,4,4,4,4,4
"Discovery Ridge, Signal Hill, West Springs, Christie Estates, Patterson, Cougar Ridge",5,5,5,5,5,5
"Douglas Glen, McKenzie Lake, Copperfield, East Shepard",8,8,8,8,8,8


In [98]:
print('There are {} uniques categories.'.format(len(calgary_venues['Venue Category'].unique())))

There are 115 uniques categories.


### Analyzing each neighborhood

In [99]:
# one hot encoding
calgary_onehot = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
calgary_onehot['Neighborhood'] = calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [calgary_onehot.columns[-1]] + list(calgary_onehot.columns[:-1])
calgary_onehot = calgary_onehot[fixed_columns]

calgary_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Café,Camera Store,Candy Store,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Smoke Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Yoga Studio
0,"Penbrooke Meadows, Marlborough",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [100]:
#grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
calgary_grouped = calgary_onehot.groupby('Neighborhood').mean().reset_index()
calgary_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Café,Camera Store,Candy Store,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Multiplex,Museum,New American Restaurant,Nightclub,Noodle House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Smoke Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Yoga Studio
0,"Braeside, Cedarbrae, Woodbine",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000,0.000000,0.111111,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brentwood, Collingwood, Nose Hill",0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bridgeland, Greenview, Zoo, YYC",0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.043478,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.0,0.000,0.000,0.000000,0.043478,0.0869

### Printing each neighborhood along with the top 5 most common venues

In [101]:
num_top_venues = 5

for hood in calgary_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = calgary_grouped[calgary_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Braeside, Cedarbrae, Woodbine----
               venue  freq
0        Hockey Rink  0.11
1           Pharmacy  0.11
2  Convenience Store  0.11
3     Ice Cream Shop  0.11
4        Pizza Place  0.11


----Brentwood, Collingwood, Nose Hill----
                     venue  freq
0         Asian Restaurant   1.0
1      American Restaurant   0.0
2        Korean Restaurant   0.0
3                Nightclub   0.0
4  New American Restaurant   0.0


----Bridgeland, Greenview, Zoo, YYC----
                       venue  freq
0       Fast Food Restaurant  0.09
1                       Bank  0.09
2       Gym / Fitness Center  0.04
3         Falafel Restaurant  0.04
4  Middle Eastern Restaurant  0.04


----City Centre, Calgary Tower----
                      venue  freq
0          Sushi Restaurant  0.07
1                    Bakery  0.07
2                       Pub  0.07
3                Restaurant  0.07
4  Mediterranean Restaurant  0.07


----Connaught, West Victoria Park----
                       ve

In [102]:
#Putting in dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [173]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = calgary_grouped['Neighborhood']

for ind in np.arange(calgary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(calgary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Braeside, Cedarbrae, Woodbine",Gym,Pharmacy,Convenience Store,Pub,Ice Cream Shop,Gas Station,Pizza Place,Coffee Shop,Hockey Rink,Eastern European Restaurant
1,"Brentwood, Collingwood, Nose Hill",Asian Restaurant,Yoga Studio,Food Court,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"Bridgeland, Greenview, Zoo, YYC",Fast Food Restaurant,Bank,Sandwich Place,Seafood Restaurant,Indian Restaurant,Noodle House,Falafel Restaurant,Restaurant,Chinese Restaurant,Convenience Store
3,"City Centre, Calgary Tower",Coffee Shop,Mediterranean Restaurant,Bakery,Sushi Restaurant,Pub,Restaurant,Brewery,Middle Eastern Restaurant,Moroccan Restaurant,Japanese Restaurant
4,"Connaught, West Victoria Park",Coffee Shop,Pub,Bar,Restaurant,Brewery,Middle Eastern Restaurant,Mediterranean Restaurant,French Restaurant,Yoga Studio,Hotel


### Clustering Neighborhoods

In [174]:
# set number of clusters
kclusters = 6

calgary_grouped_clustering = calgary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 2, 2, 2, 0, 2, 2, 2, 2])

In [167]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

calgary_merged = calgary_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
calgary_merged = calgary_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

calgary_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320,5,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner
1,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,2,Convenience Store,Asian Restaurant,Gas Station,Café,Flea Market,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
2,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786,2,Smoke Shop,Bar,Convenience Store,Food Court,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,T3B,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",51.0809,-114.1616,0,Food Court,Coffee Shop,Bank,Scenic Lookout,Flea Market,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001,2,Convenience Store,Pizza Place,Diner,Clothing Store,Yoga Studio,Deli / Bodega,Department Store,Dim Sum Restaurant,Discount Store,Dog Run


In [175]:
#Visualizing the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(calgary_merged['Latitude'], calgary_merged['Longitude'], calgary_merged['Neighborhood'], calgary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [176]:
cluster1_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 0, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster1_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Montgomery, Bowness, Silver Springs, Greenwood",0,Food Court,Coffee Shop,Bank,Scenic Lookout,Flea Market,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
15,"Sandstone, MacEwan Glen, Beddington, Harvest H...",0,Pizza Place,Grocery Store,Italian Restaurant,Liquor Store,Pharmacy,Bank,Coffee Shop,Dog Run,Dry Cleaner,Donut Shop
17,"Tuscany, Scenic Acres",0,Pizza Place,Pharmacy,Liquor Store,Pub,Chinese Restaurant,Convenience Store,Video Store,Diner,Dim Sum Restaurant,Eastern European Restaurant
19,"Cranston, Auburn Bay, Mahogany",0,Cosmetics Shop,Pizza Place,Liquor Store,Fast Food Restaurant,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
28,South Calgary (Altadore / Bankview / Richmond),0,Convenience Store,Pizza Place,Coffee Shop,Liquor Store,Yoga Studio,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
31,"Midnapore, Sundance",0,Mobile Phone Shop,Liquor Store,Hardware Store,Yoga Studio,Fast Food Restaurant,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store


### Cluster 2

In [177]:
cluster2_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 1, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster2_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Northwest Calgary,1,Flea Market,Yoga Studio,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


### Cluster 3

In [178]:
cluster3_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 2, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster3_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",2,Convenience Store,Asian Restaurant,Gas Station,Café,Flea Market,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
2,"Forest Lawn, Dover, Erin Woods",2,Smoke Shop,Bar,Convenience Store,Food Court,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",2,Convenience Store,Pizza Place,Diner,Clothing Store,Yoga Studio,Deli / Bodega,Department Store,Dim Sum Restaurant,Discount Store,Dog Run
5,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",2,Sandwich Place,Pub,Indian Restaurant,Mexican Restaurant,Gas Station,Sports Bar,Pizza Place,Vietnamese Restaurant,Fast Food Restaurant,Breakfast Spot
6,"Bridgeland, Greenview, Zoo, YYC",2,Fast Food Restaurant,Bank,Sandwich Place,Seafood Restaurant,Indian Restaurant,Noodle House,Falafel Restaurant,Restaurant,Chinese Restaurant,Convenience Store
7,"Lakeview, Glendale, Killarney, Glamorgan",2,Convenience Store,Coffee Shop,Wine Shop,Arts & Crafts Store,Food Truck,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
8,"Inglewood, Burnsland, Chinatown, East Victoria...",2,Hotel,Pub,Coffee Shop,Performing Arts Venue,Theater,Cocktail Bar,Restaurant,Asian Restaurant,New American Restaurant,Italian Restaurant
10,"Highfield, Burns Industrial",2,American Restaurant,Gym,Karaoke Bar,Gym / Fitness Center,Grocery Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
11,"Discovery Ridge, Signal Hill, West Springs, Ch...",2,Gas Station,Convenience Store,Vietnamese Restaurant,Pizza Place,Bar,Yoga Studio,Eastern European Restaurant,Dry Cleaner,Donut Shop,Dog Run
12,"Queensland, Lake Bonavista, Willow Park, Acadia",2,Pizza Place,Chinese Restaurant,Child Care Service,Insurance Office,Yoga Studio,Fast Food Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store


### Cluster 4

In [179]:
cluster4_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 3, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster4_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Martindale, Taradale, Falconridge, Saddle Ridge",3,Dog Run,Yoga Studio,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dry Cleaner


### Cluster 5

In [180]:
cluster5_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 4, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster5_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Brentwood, Collingwood, Nose Hill",4,Asian Restaurant,Yoga Studio,Food Court,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


### Cluster 6

In [185]:
cluster6_df = calgary_merged.loc[calgary_merged['Cluster Labels'] == 5, calgary_merged.columns[[2] + list(range(5, calgary_merged.shape[1]))]]
cluster6_df

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Penbrooke Meadows, Marlborough",5,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner
